# Image Data Augmentation Exercise : CIFAR-100
---
## 데이터가 더 부족한 세상으로!

[여기 참고](https://www.cs.toronto.edu/~kriz/cifar.html)

## Data Loading

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.datasets.cifar100 import load_data

In [ ]:
(train_x, train_y), (test_x, test_y) = load_data()
# (train_x, train_y), (test_x, test_y) = load_data(label_mode='coarse')

In [ ]:
np.unique(train_y)

In [ ]:
label_dict = {0:'apple', 1: 'aquarium_fish', 2: 'baby', 3: 'bear', 4: 'beaver', 5: 'bed', 6: 'bee', 7: 'beetle', 8: 'bicycle', 9: 'bottle',
              10: 'bowl', 11: 'boy',12: 'bridge',13: 'bus',14: 'butterfly',15: 'camel',16: 'can',17: 'castle',18: 'caterpillar',19: 'cattle',
              20: 'chair',21: 'chimpanzee',22: 'clock',23: 'cloud',24: 'cockroach',25: 'couch',26: 'cra',27: 'crocodile',28: 'cup',29: 'dinosaur',
              30: 'dolphin',31: 'elephant',32: 'flatfish',33: 'forest',34: 'fox',35: 'girl',36: 'hamster',37: 'house',38: 'kangaroo',39: 'keyboard',
              40: 'lamp',41: 'lawn_mower',42: 'leopard',43: 'lion',44: 'lizard',45: 'lobster',46: 'man',47: 'maple_tree',48: 'motorcycle',49: 'mountain',
              50: 'mouse',51: 'mushroom',52: 'oak_tree',53: 'orange',54: 'orchid',55: 'otter',56: 'palm_tree',57: 'pear',58: 'pickup_truck',59: 'pine_tree',
              60: 'plain',61: 'plate',62: 'poppy',63: 'porcupine',64: 'possum',65: 'rabbit',66: 'raccoon',67: 'ray',68: 'road',69: 'rocket',
              70: 'rose',71: 'sea',72: 'seal',73: 'shark',74: 'shrew',75: 'skunk',76: 'skyscraper',77: 'snail',78: 'snake',79: 'spider',
              80: 'squirrel',81: 'streetcar',82: 'sunflower',83: 'sweet_pepper',84: 'table',85: 'tank',86: 'telephone',87: 'television',88: 'tiger',89: 'tractor',
              90: 'train',91: 'trout',92: 'tulip',93: 'turtle',94: 'wardrobe',95: 'whale',96: 'willow_tree',97: 'wolf',98: 'woman',99: 'worm'
            }

label_dict[0]

* 데이터 살펴보기

In [ ]:
rand_i = np.random.randint(0, train_x.shape[0])

plt.title(f'idx: {rand_i} , class: { label_dict[train_y[rand_i][0]] }')
plt.imshow( train_x[rand_i] )
plt.show()

In [ ]:
rows = 5
fig, axes = plt.subplots(rows, len(label_dict), figsize=(len(label_dict), rows) )

for img_id in range(len(label_dict)) :
    imgs = train_x[train_y.reshape(-1)==img_id]
    imgs_len = len(imgs)

    for row_i in range(rows) :
        axe = axes[row_i, img_id]
        axe.imshow( imgs[np.random.randint(imgs_len)], interpolation='none' )
        axe.axis('off')

plt.tight_layout()
plt.show()

## Data Preprocessing

* Data split
    - training set : test set = 8 : 2
    - training set : validation set = 8 : 2
    - 재현을 위한 난수 고정 : 2023

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(train_x, train_y, test_size=0.2, random_state=2023)
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=2023)

In [ ]:
train_x.shape, train_y.shape

* Scaling

    - min-max scaling

In [ ]:
tr_r_max, tr_r_min = train_x[:,:,:,0].max(), train_x[:,:,:,0].min()
tr_g_max, tr_g_min = train_x[:,:,:,1].max(), train_x[:,:,:,1].min()
tr_b_max, tr_b_min = train_x[:,:,:,2].max(), train_x[:,:,:,2].min()

In [ ]:
tr_r_mm = (train_x[:,:,:,0] - tr_r_min) / (tr_r_max - tr_r_min)
tr_g_mm = (train_x[:,:,:,1] - tr_g_min) / (tr_g_max - tr_g_min)
tr_b_mm = (train_x[:,:,:,2] - tr_b_min) / (tr_b_max - tr_b_min)

In [ ]:
train_x_mm = np.stack((tr_r_mm, tr_g_mm, tr_b_mm), axis=3)

In [ ]:
val_r_mm = (val_x[:,:,:,0] - tr_r_min) / (tr_r_max - tr_r_min)
val_g_mm = (val_x[:,:,:,1] - tr_g_min) / (tr_g_max - tr_g_min)
val_b_mm = (val_x[:,:,:,2] - tr_b_min) / (tr_b_max - tr_b_min)

In [ ]:
val_x_mm = np.stack((val_r_mm, val_g_mm, val_b_mm), axis=3)

In [ ]:
te_r_mm = (test_x[:,:,:,0] - tr_r_min) / (tr_r_max - tr_r_min)
te_g_mm = (test_x[:,:,:,1] - tr_g_min) / (tr_g_max - tr_g_min)
te_b_mm = (test_x[:,:,:,2] - tr_b_min) / (tr_b_max - tr_b_min)

In [ ]:
test_x_mm = np.stack((te_r_mm, te_g_mm, te_b_mm), axis=3)

In [ ]:
train_x_mm[:,:,:,0].max(), train_x_mm[:,:,:,0].min()

In [ ]:
train_x_mm.shape

* One-hot encoding

In [ ]:
class_n = len(np.unique(train_y))
class_n

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
train_y = to_categorical(train_y, class_n)
val_y = to_categorical(val_y, class_n)
test_y = to_categorical(test_y, class_n)

* Data shape 재확인

In [ ]:
train_x.shape, train_y.shape

## **Image Data Augmentation**

- ImageDataGenerator : [**전체 옵션 참고**](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)
- .flow( )

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
trainIDG = ImageDataGenerator(rotation_range=20,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.15,
                               zoom_range=0.15,
                               horizontal_flip=False,
                               vertical_flip=True)

valIDG = ImageDataGenerator()

In [ ]:
flow_trainIDG = trainIDG.flow(train_x, train_y)
flow_valIDG = valIDG.flow(val_x, val_y)

## Modeling : CNN

- 조건
    1. Sequential API, Functiona API 중 택일.
    2. [이 구조를 미니 버전으로 활용해봐도 좋다.](https://img1.daumcdn.net/thumb/R800x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F99DFA5415B38AC752E)
    3. DropOut, BatchNormalization 등의 기능도 같이 활용해보자.
    4. Early Stopping을 사용할 것.

In [ ]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.layers import Conv2D, MaxPool2D

* Sequential API

In [ ]:
# 1. 세션 클리어
clear_session()

# 2. 모델 선언
model1 = Sequential()

# 3. 레이어 블록 조립
model1.add( Input(shape=(32,32,3)) )

model1.add( Conv2D(filters=64,          # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
                   kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
                   strides=(1,1),       # Conv2D 필터의 이동 보폭
                   padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
                   activation='relu',   # 빼먹지 않게 주의!
                   ) )
model1.add( Conv2D(filters=64,          # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
                   kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
                   strides=(1,1),       # Conv2D 필터의 이동 보폭
                   padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
                   activation='relu',   # 빼먹지 않게 주의!
                   ) )
model1.add( MaxPool2D(pool_size=(2,2),  # Maxpool2D 필터의 가로 세로 사이즈
                      strides=(2,2)     # Maxpool2D 필터의 이동 보폭 : 기본적으로 pool_size를 따른다.
                      ) )
model1.add( BatchNormalization() )
model1.add( Dropout(0.2) )

model1.add( Conv2D(filters=128,         # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
                   kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
                   strides=(1,1),       # Conv2D 필터의 이동 보폭
                   padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
                   activation='relu',   # 빼먹지 않게 주의!
                   ) )
model1.add( Conv2D(filters=128,         # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
                   kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
                   strides=(1,1),       # Conv2D 필터의 이동 보폭
                   padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
                   activation='relu',   # 빼먹지 않게 주의!
                   ) )
model1.add( MaxPool2D(pool_size=(2,2),  # Maxpool2D 필터의 가로 세로 사이즈
                      strides=(2,2)     # Maxpool2D 필터의 이동 보폭 : 기본적으로 pool_size를 따른다.
                      ) )
model1.add( BatchNormalization() )
model1.add( Dropout(0.2) )

model1.add( Flatten() )
model1.add( Dense(256, activation='relu') )
model1.add( Dense(100, activation='softmax') )

# 4. 컴파일
model1.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

model1.summary()

* Functional API

In [ ]:
# 1. 세션 클리어
clear_session()

# 2. 레이어 엮기
il = Input(shape=(32,32,3))

hl = Conv2D(filters=64,          # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
            kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
            strides=(1,1),       # Conv2D 필터의 이동 보폭
            padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
            activation='relu',   # 빼먹지 않게 주의!
            )(il)
hl = Conv2D(filters=64,          # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
            kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
            strides=(1,1),       # Conv2D 필터의 이동 보폭
            padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
            activation='relu',   # 빼먹지 않게 주의!
            )(hl)
hl = MaxPool2D(pool_size=(2,2),  # Maxpool2D 필터의 가로 세로 사이즈
               strides=(2,2)     # Maxpool2D 필터의 이동 보폭 : 기본적으로 pool_size를 따른다.
               )(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)

hl = Conv2D(filters=128,         # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
            kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
            strides=(1,1),       # Conv2D 필터의 이동 보폭
            padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
            activation='relu',   # 빼먹지 않게 주의!
            )(hl)
hl = Conv2D(filters=128,         # Conv2D 필터를 통해 새롭게 제작하려는 feature map의 수
            kernel_size=(3,3),   # Conv2D 필터의 가로 세로 사이즈
            strides=(1,1),       # Conv2D 필터의 이동 보폭
            padding='same',      # 1. 기존의 사이즈를 보존하겠다. | 2. 외곽의 정보를 조금 더 반영하려고!
            activation='relu',   # 빼먹지 않게 주의!
            )(hl)
hl = MaxPool2D(pool_size=(2,2),  # Maxpool2D 필터의 가로 세로 사이즈
               strides=(2,2)     # Maxpool2D 필터의 이동 보폭 : 기본적으로 pool_size를 따른다.
               )(hl)
hl = BatchNormalization()(hl)
hl = Dropout(0.2)(hl)

hl = Flatten()(hl)
hl = Dense(256, activation='relu')(hl)
ol = Dense(100, activation='softmax')(hl)

# 3. 모델의 시작과 끝 지정
model2 = Model(il, ol)

# 4. 컴파일
model2.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

model2.summary()

* Early Stopping

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
es = EarlyStopping(monitor='val_loss',          # 얼리 스토핑을 적용할 관측 대상
                   min_delta=0,                 # Threshold. 설정한 값 이상으로 변해야 성능 개선되었다고 간주.
                   patience=5,                  # 성능 개선이 발생하지 않았을 때, 몇 epoch를 더 지켜볼 것인가.
                   verbose=1,
                   restore_best_weights=True)   # 성능이 가장 좋은 epoch의 가중치를 적용함.

* .fit( )
    - Data Augmentation 과정에서 생성한 ImageDataGenerator를 사용해야 한다.

In [ ]:
model1.fit(flow_trainIDG,               # 위에서 설정한 ImageDataGenerator를 사용해야 한다!
           epochs=20, verbose=1,
           validation_data=flow_valIDG, # validation set 역시 ImageDataGenerator를 사용!
           callbacks=[es]               # 얼리스토핑 적용
           )

* .evaluate( )

In [ ]:
model1.evaluate(test_x, test_y, verbose=1)

* .predict( )

In [ ]:
y_pred = model1.predict(test_x)

In [ ]:
# 원핫 인코딩 한 것을 다시 묶어주는 코드
# 평가 지표 및 실제 데이터 확인을 위해 필요

y_pred_arg = np.argmax(y_pred, axis=1)
test_y_arg = np.argmax(test_y, axis=1)

* 평가 지표

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score(test_y_arg, y_pred_arg)

In [ ]:
print( classification_report(test_y_arg, y_pred_arg, target_names=list(label_dict.values())) )

## Visualization

* 실제 데이터 확인

In [ ]:
rand_idx = np.random.randint(0, len(y_pred_arg))
test_idx = test_y_arg[rand_idx]
pred_idx = y_pred_arg[rand_idx]
class_prob = np.floor( y_pred[rand_idx]*100 )

print(f'idx = {rand_idx}')
print(f'해당 인덱스의 이미지는 {label_dict[test_idx]}')
print(f'모델의 예측 : {label_dict[pred_idx]}')
print(f'모델의 클래스별 확률 : ')
print('-------------------')
for idx, val in enumerate( list(label_dict.values()) ) :
    print(val, class_prob[idx])
print('=================================================')

if test_y_arg[rand_idx] == y_pred_arg[rand_idx] :
    print('정답')
else :
    print('땡')

plt.imshow(test_x[rand_idx])
plt.show()

* 틀린 이미지만 확인해보기

In [ ]:
temp = (test_y_arg == y_pred_arg)
false_idx = np.where(temp==False)[0]
false_len = len(false_idx)
false_len

In [ ]:
rand_idx = false_idx[np.random.randint(0, false_len)]
test_idx = test_y_arg[rand_idx]
pred_idx = y_pred_arg[rand_idx]
class_prob = np.floor( y_pred[rand_idx]*100 )

print(f'idx = {rand_idx}')
print(f'해당 인덱스의 이미지는 {label_dict[test_idx]}')
print(f'모델의 예측 : {label_dict[pred_idx]}')
print(f'모델의 클래스별 확률 : ')
print('-------------------')
for idx, val in enumerate( list(label_dict.values()) ) :
    print(val, class_prob[idx])
print('=================================================')

if test_y_arg[rand_idx] == y_pred_arg[rand_idx] :
    print('정답')
else :
    print('땡')

plt.imshow(test_x[rand_idx] )
plt.show()